This example demonstrates how to construct FSM's in magma using the combinational circuit definition.

# UART
[UART](https://en.wikipedia.org/wiki/Universal_asynchronous_receiver-transmitter) is a serialization/deserialization protocol. It consists of a transmitter circuit and reciever circuit. Let's take a look at a verilog implementation of the transmitter circuit.

```verilog
module TXMOD(
  output TX,
  input CLK,
  input [7:0] data,
  input valid,
  output ready);

reg TXReg = 1;
assign TX = TXReg;

reg [10:0] dataStore = 1536; // MSB=1, LSB=0
reg writing = 0;
assign ready = (writing==0);

reg [13:0] writeClock = 0; // which cycle are we in?
reg [3:0] writeBit = 0; // which bit are we writing? (10 bits total)

always @(posedge CLK) begin
  if(writing==0 && valid==1) begin
    writing <= 1;
    dataStore[8:1] <= data;
    writeClock <= 100;
    writeBit <= 0;
    TXReg <= dataStore[0];
  end else if(writing==1 && writeClock==0 && writeBit==9) begin
    // we're done
    TXReg <= 1;
    writing <= 0;
  end else if(writing==1 && writeClock==0) begin
    // move on to next bit
    TXReg <= dataStore[writeBit];
    writeBit <= writeBit+1;
    writeClock <= 100;
  end else if(writing==1) begin
    TXReg <= dataStore[writeBit];
    writeClock <= writeClock - 1;
  end else begin
    TXReg <= 1;
  end 

end

endmodule
```

We can see that it is essentially a simple FSM. Using magma's [circuit.combinational](https://github.com/phanrahan/magma/blob/master/docs/circuit_definitions.md#combinational-circuit-definitions) we can replicate this functionality.

In [1]:
import magma as m
m.set_mantle_target("ice40")
from mantle import Register


@m.circuit.combinational
def txmod_logic(
        data : m.Bits(8),
        writing : m.Bit,
        valid : m.Bit,
        dataStore : m.Bits(11),
        writeClock : m.Bits(14),
        writeBit : m.Bits(4),) -> (m.Bit,
                                   m.Bits(11),
                                   m.Bits(14),
                                   m.Bits(4),
                                   m.Bit,):

    if (writing == m.bit(0)) & (valid == m.bit(1)):
        writing_out = m.bit(1)
        dataStore_out = m.concat(dataStore[0:1], data, dataStore[9:])
        writeClock_out = m.bits(100, 14)
        writeBit_out = m.bits(0, 4)
        TXReg_out = dataStore[0]
    elif (writing == m.bit(1)) & \
         (writeClock == m.bits(0, 14)) & \
         (writeBit == m.bits(9, 4)):
        dataStore_out = dataStore
        writeClock_out = writeClock
        writeBit_out = writeBit
        TXReg_out = m.bit(1)
        writing_out = m.bit(0)
    elif (writing == m.bit(1)) & (writeClock == m.bits(0, 14)):
        writing_out = writing
        dataStore_out = dataStore
        TXReg_out = dataStore[writeBit]
        writeBit_out = m.bits(m.uint(writeBit) + m.bits(1, 4))
        writeClock_out = m.bits(100, 14)
    elif writing == m.bit(1):
        writing_out = writing
        dataStore_out = dataStore
        writeBit_out = writeBit
        TXReg_out = dataStore[writeBit]
        writeClock_out = m.bits(m.uint(writeClock) - m.bits(1, 14))
    else:
        writing_out = writing
        dataStore_out = dataStore
        writeClock_out = writeClock
        writeBit_out = writeBit
        TXReg_out = m.bit(1)

    return (writing_out,
            dataStore_out,
            writeClock_out,
            writeBit_out,
            TXReg_out,)


class TXMOD(m.Circuit):
    IO = ["TX", m.Out(m.Bit),
          "data", m.In(m.Bits(8)),
          "valid", m.In(m.Bit),
          "ready", m.Out(m.Bit),
          "CLK", m.In(m.Clock),]

    @classmethod
    def definition(io):
        TXReg = Register(1, init=1)
        dataStore = Register(11, init=1536)
        writing = Register(1, init=0)
        writeClock = Register(14, init=0)
        writeBit = Register(4, init=0)
        (writing_next,
         dataStore_next,
         writeClock_next,
         writeBit_next,
         TXReg_next,) = txmod_logic(io.data,
                                   writing.O[0],
                                   io.valid,
                                   dataStore.O,
                                   writeClock.O,
                                   writeBit.O)
        m.wire(writing_next, writing.I[0])
        m.wire(dataStore_next, dataStore.I)
        m.wire(writeClock_next, writeClock.I)
        m.wire(writeBit_next, writeBit.I)
        m.wire(TXReg_next, TXReg.I[0])
        ready = writing.O[0] == m.bit(0)
        m.wire(ready, io.ready)
        m.wire(TXReg.O[0], io.TX)

import lattice ice40
import lattice mantle40


We replicate the functionality by splitting up the logic into a *purely* combinational part and a stateful part. The function `txmod_logic()` which is decorated with `m.circuit.combinational`, is converted into a combinational circuit which describes the next state as a function of the current state.

## Limitations
Note that `circuit.combinational` has some limitations. First, default values for outputs is not supported, therefore each output must be set in each `if`/`else-if`/`else` block. This adds many more lines of code compared to the verilog version. Also, temporaries and over-writing/reusing variables within the combinational function is not supported. This limits the readability and compactness of the code.

## Using the TXMOD circuit
We can now use the `TXMOD` circuit with the ice stick. We will first do this using the verilog implementations (see `txmod.v`, `rxmod.v`, and `uart_main.v`). The main circuit reads in a stream of characters and returns the same stream offset by 10 (i.e. +10 in ascii).

Note that here and below, you may need to do some setup to get the icesticks working, see the previous tutorials for directions. Also, you may need to replace `/dev/tty.usbserial-14101` with the actual path of your mounted icestick, wherever it appears below.

In [2]:
%%bash
make clean && make uart_main.run

\rm -rf out/*
mkdir -p out
yosys -q -p 'synth_ice40 -top main -blif out/uart_main.blif' txmod.v rxmod.v uart_main.v
arachne-pnr -q -d 1k -o out/uart_main_pnr.txt -p ice40.pcf out/uart_main.blif
icepack out/uart_main_pnr.txt out/uart_main.bin
iceprog out/uart_main.bin
rm out/uart_main_pnr.txt out/uart_main.blif


init..
cdone: high
reset..
cdone: low
flash ID: 0x20 0xBA 0x16 0x10 0x00 0x00 0x23 0x51 0x73 0x10 0x23 0x00 0x35 0x00 0x35 0x06 0x06 0x15 0x43 0xB6
file size: 32220
erase 64kB sector at 0x000000..
programming..
reading..
VERIFY OK
cdone: high
Bye.


In [3]:
%%bash
python uart_driver.py infile outfile /dev/tty.usbserial-14101

Running with infile=infile, outfile=outfile
infile length=12


In [4]:
%cat outfile

rovvy*�y|vn

Now let's replace the verilog version with the magma version of the `TXMOD` circuit (which will overwrite `txmod.v`).

In [5]:
%%bash
python txmod.py

import lattice ice40
import lattice mantle40


In [6]:
m.compile("txmod", TXMOD, output="verilog")

In [7]:
%cat txmod.v

module Register1_0001 (input [0:0] I, output [0:0] O, input  CLK);
wire  inst0_Q;
wire  inst1_O;
wire  inst2_O;
SB_DFF inst0 (.C(CLK), .D(inst1_O), .Q(inst0_Q));
SB_LUT4 #(.LUT_INIT(16'h5555)) inst1 (.I0(I[0]), .I1(1'b0), .I2(1'b0), .I3(1'b0), .O(inst1_O));
SB_LUT4 #(.LUT_INIT(16'h5555)) inst2 (.I0(inst0_Q), .I1(1'b0), .I2(1'b0), .I3(1'b0), .O(inst2_O));
assign O = {inst2_O};
endmodule

module Register11_0600 (input [10:0] I, output [10:0] O, input  CLK);
wire  inst0_Q;
wire  inst1_Q;
wire  inst2_Q;
wire  inst3_Q;
wire  inst4_Q;
wire  inst5_Q;
wire  inst6_Q;
wire  inst7_Q;
wire  inst8_Q;
wire  inst9_Q;
wire  inst10_O;
wire  inst11_O;
wire  inst12_Q;
wire  inst13_O;
wire  inst14_O;
SB_DFF inst0 (.C(CLK), .D(I[0]), .Q(inst0_Q));
SB_DFF inst1 (.C(CLK), .D(I[1]), .Q(inst1_Q));
SB_DFF inst2 (.C(CLK), .D(I[2]), .Q(inst2_Q));
SB_DFF inst3 (.C(CLK), .D(I[3]), .Q(inst3_Q));
SB_DFF inst4 (.C(CLK), .D(I[4]), .Q(inst4_Q));
SB_DFF inst5 (.C(CLK), .D(I[5]), .Q(inst5_Q));
SB_DFF inst6 (.C(CLK), .D(I[

Now we run the flow again, and see that the output has not changed!

In [8]:
%%bash
make clean && make uart_main.run
python uart_driver.py infile outfile /dev/tty.usbserial-14101

\rm -rf out/*
mkdir -p out
yosys -q -p 'synth_ice40 -top main -blif out/uart_main.blif' txmod.v rxmod.v uart_main.v
arachne-pnr -q -d 1k -o out/uart_main_pnr.txt -p ice40.pcf out/uart_main.blif
icepack out/uart_main_pnr.txt out/uart_main.bin
iceprog out/uart_main.bin
rm out/uart_main_pnr.txt out/uart_main.blif
Running with infile=infile, outfile=outfile
infile length=12


init..
cdone: high
reset..
cdone: low
flash ID: 0x20 0xBA 0x16 0x10 0x00 0x00 0x23 0x51 0x73 0x10 0x23 0x00 0x35 0x00 0x35 0x06 0x06 0x15 0x43 0xB6
file size: 32220
erase 64kB sector at 0x000000..
programming..
reading..
VERIFY OK
cdone: high
Bye.


In [10]:
%cat outfile

rovvy*�y|vn